# Dash

## Prepare dataset

Calculate the weights of the classes relative to the sample size

In [6]:
from pathlib import Path

raw_data_path = Path('data/KWS_DASH/raw')
class_file_count = {}

class_dirs = [d for d in raw_data_path.iterdir() if d.is_dir() and d.stem != "_background_noise_"]


for d in class_dirs:
    class_file_count[d] = len(list(d.iterdir()))

for d in class_dirs:
    class_file_count[d] = class_file_count[d] / float(max(class_file_count.values()))
    print(f"{d.stem}: {round(class_file_count[d], 3)}")

blinds: 0.008
dash: 0.008
down: 1.0
energy: 0.008
lights: 0.008
off: 0.974
on: 1.0
up: 1.0


Generate processed dataset

In [8]:
import datasets.kws20 as kws20
kws20.KWS_DASH_get_datasets(('data', ''))

No key `noise_var` in input augmentation dictionary!  Using defaults: [Min: 0., Max: 1.]
No key `shift` in input augmentation dictionary! Using defaults: [Min:-0.1, Max: 0.1]
No key `strech` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]
Generating dataset from raw data samples for the first time. 
This process will take significant time (~60 minutes)...
data_len: 16384
------------- Label Size ---------------
blinds  :  	31
dash    :  	31
down    :  	3917
energy  :  	32
lights  :  	32
off     :  	3745
on      :  	3845
up      :  	3723
------------------------------------------
Processing the label: blinds. 1 of 8
	1 of 31
Finished in 1.108 seconds.
(93, 128, 128)
Data concatenation finished in 0.000 seconds.
Processing the label: dash. 2 of 8
	1 of 31
Finished in 1.063 seconds.
(93, 128, 128)
Data concatenation finished in 0.016 seconds.
Processing the label: down. 3 of 8
	1 of 3917
	1001 of 3917
	2001 of 3917
	3001 of 3917
Finished in 134.412 seconds.
(11751, 

(<datasets.kws20.KWS_DASH at 0x16fb5298be0>,
 <datasets.kws20.KWS_DASH at 0x16fb524cc40>)